# Models evaluations

This kaggle notebook is used to evaluate the different models that have been trained. Its outputs can then serve to make comparisions and see how they perform, what parameters were used, the number of epochs or the data augmentation policy.

Combined with tensorboard plots, it will also be used as the source for reporting some of the results in the final memory to deliver.

In order to get this notebook working in a cloud platform like Kaggle, we need:

1. The datatet to the slices (IXI-T1-slices)
2. The dataset or datasets to results, where .h5 NN weight and .hostory files the exist. Usually, these datasets also contain a logs folder with tensorboard data that is interpreted in Google Colab notebook (since kaggle seems to have some issues with tensorboard support)

In [1]:
import os
import numpy as np
import pandas as pd
import glob
import imageio
import random
import tensorflow as tf
import matplotlib.pyplot as plt
import math
from keras import layers
import tensorflow_addons as tfa
import pickle

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1500)
pd.set_option('display.colheader_justify', 'left')
pd.set_option('display.precision', 3)

We need the Augmentation class to pass corrupted slices to the models and see how the results look:

In [2]:
class Augmentation():
    """
    Helper to apply some filters to images
    """
    def random(self, images):
        # Each filter is applied with a probability of 25%, except cutout which is applied half of the times
        if random.randint(1, 4) == 1:
            images = self.add_noise(images)
        if random.randint(1, 4) == 1:
            images = self.dropout(images)
        if random.randint(1, 4) == 1:
            images = self.gaussian_blur(images)
        if random.randint(1, 2) == 1:
            images = self.cutout(images)
        return images

    def add_noise(self, images):
        sdev = 0 + (random.random() * (0.04 - 0))
        images = layers.GaussianNoise(stddev=sdev)(images, training=True)
        return images

    def dropout(self, images):
        rnds_noise = tf.random.uniform((1, 2), minval=0, maxval=0.04)
        images = tf.nn.dropout(images, rnds_noise[0][0])
        return images

    # https://www.tensorflow.org/addons/api_docs/python/tfa/image/gaussian_filter2d
    def gaussian_blur(self, images):
        images = tfa.image.gaussian_filter2d(images,
                                             filter_shape=[4, 4],
                                             sigma=0.8,
                                             constant_values=0,
                                             )
        return images

    def cutout(self, images):
        w = tf.random.uniform((), minval=10, maxval=20, dtype=tf.dtypes.int32)
        h = tf.random.uniform((), minval=10, maxval=20, dtype=tf.dtypes.int32)
        x = tf.random.uniform((), minval=20, maxval=105, dtype=tf.dtypes.int32)
        y = tf.random.uniform((), minval=40, maxval=105, dtype=tf.dtypes.int32)

        if w % 2 != 0:
            w += 1 if bool(random.getrandbits(1)) else -1
        if h % 2 != 0:
            h += 1 if bool(random.getrandbits(1)) else -1

        # image = tfa.image.random_cutout(image, mask_size=(w,h), constant_values=0)
        images = tfa.image.cutout(images,
                                  mask_size=(w, h),
                                  offset=(x, y),
                                  constant_values=0
                                  )
        return images

The next functions help the script visualize the results:

In [3]:
def get_random_images(mode="skull-stripped", set='test', n=20):
    """
    returns a randomly sorted list of numpy slices from a subset (test set by default)
    """
    base_dir = f'../input/ixit1slices/IXI-T1-slices/{mode}/{set}'
    files = glob.glob(f'{base_dir}/*.png')
    random.shuffle(files)
    files = files[0:n]
    imgs = []
    for file in files:
        im = imageio.imread(file)
        # prepare data with the same shape the NNs were fed
        im = im.astype(np.float32)
        im = im / 255
        im = im.reshape(256, 256, 1)
        imgs.append(im)
    
    imgs = tf.image.resize(imgs, [128, 128]).numpy()
    return imgs

def show_img_grid(imgs, ncols=5, show_axis=False):
    """
    shows a grid with ncols columns with an element in img for each resulting cell
    """
    nrows = math.ceil(len(imgs) / ncols)
    f, axarr = plt.subplots(nrows, ncols, figsize=(4*ncols,4*nrows))
    i = 0
       
    for img in imgs:
        cur_row = i // ncols
        cur_col = i % ncols
        if nrows == 1:
            axarr[cur_col].imshow(np.rot90(img), cmap="gray")
            if not show_axis:
                axarr[cur_col].axis('off')
        else:
            axarr[cur_row, cur_col].imshow(np.rot90(img), cmap="gray")
            if not show_axis:
                axarr[cur_row, cur_col].axis('off')
        i += 1
    
def predict_imgs(inputs, model, augmentation=[]):
    """
    predicts the input images and returns a list with pairs of images in consecutive order
    to easily plot them
    """
    if len(augmentation) > 0:
        augment = Augmentation()
        augmented = inputs
        if "noise" in augmentation:
            augmented = augment.add_noise(augmented)
        if "blur" in augmentation:
            augmented = augment.gaussian_blur(augmented)
        if "dropout" in augmentation:
            augmented = augment.dropout(augmented)
        if "cutout" in augmentation:
            augmented = augment.cutout(augmented)
        if "random" in augmentation:
            augmented = augment.random(augmented)
        predicted = model.predict(augmented)
        outputs = [None]*(len(inputs)*3)        
        outputs[::3] = inputs
        outputs[1::3] = augmented
        outputs[2::3] = predicted        
        return outputs
    else:        
        predicted = model.predict(inputs)
        outputs = [None]*(len(inputs)*2)
        outputs[::2] = inputs
        outputs[1::2] = predicted
        return outputs
    
def show_history_data(path, show_table=True, plot=True):
    """
    opens a history file generated by a model's training process. It is also possible to see the loss graphs 
    when de param. plot equals True
    """
    with open(path, "rb") as input_file:
        history = pickle.load(input_file)
        
    if show_table:
        l = []
        for i in range(0, len(history['loss'])):
            l.append({'Epoch': i + 1, 'loss': history['loss'][i], 'val_loss': history['val_loss'][i] })        
        print(pd.DataFrame(l))
        
    if plot:    
        plt.style.use("ggplot")
        plt.plot(np.arange(0, len(history['loss'])), history["loss"], label="train")
        plt.plot(np.arange(0, len(history['val_loss'])), history["val_loss"], label="val")
        plt.title("Loss")
        plt.xlabel("Epoch #")
        plt.ylabel("Loss")
        plt.legend(loc="upper right")
        plt.show()


# Showing a training history
We can show a training history like this:

In [4]:
path = '../input/newgenerator/model_baseline_10epoch_es20_activation_sigmoid.h12.history'
show_history_data(path)

# Qualitative results
And qualitative results like this:

In [6]:
from keras.models import load_model
imgs = get_random_images(n=8)
path_to_model = "../input/newgenerator/model_baseline_10epoch_es4_activation_sigmoid.h5"
model = load_model(path_to_model)
outputs = predict_imgs(imgs, model, ['dropout'])
show_img_grid(outputs, ncols=3)